# Bagging

1. 데이터 준비 & 전처리

* Breast Cancer Wisconsin (Diagnostic) 데이터셋을 로딩한다

* StandardScaler로 피처별 평균 0, 분산 1로 표준화한다

2. 차원 축소 (PCA)

* 전체 분산의 90%를 보존하는 주성분만 추출해 차원을 축소한다

* 상관성이 높은 원본 변수들을 서로 직교하는 축으로 통합한다

3. Bagging 분류기 하이퍼파라미터 튜닝 (OOB 기반 그리드 탐색)

* 트리 수(n_estimators), Bootstrap 샘플 비율(max_samples), 특성 비율(max_features) 조합 12가지 test한다

* 각 조합별로 Out-Of-Bag(OOB) 스코어를 계산하여 최적 설정을 탐색한다

4. 최적 모델 학습

* 찾은 하이퍼파라미터로 BaggingClassifier를 재생성·학습한다

* OOB 평가를 통해 학습 성능을 검증한다

5. MDI 중요도 계산 (PCA 축 → 원본 변수)

* 각 트리가 본 PCA 축만큼만 계산한 feature_importances_를 전체 축 길이로 매핑하여 평균한다

* PCA 로딩(loadings) 행렬과 곱해 원본 30개 피처 수준의 중요도를 근사한다

6. 중요 결과를 표로 출력

* 하이퍼파라미터별 OOB 결과 표를 출력한다

* 원본 변수 중요도 순위 표를 출력한다

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer   # UCI 저장소의 Breast Cancer Wisconsin (Diagnostic) 데이터셋을 불러오는 함수
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
# UCI 저장소의 Breast Cancer Wisconsin (Diagnostic) 데이터셋을 loading해서 data 변수공간에 저장
data = load_breast_cancer()

In [4]:
type(data)

sklearn.utils._bunch.Bunch

In [5]:
data.DESCR

'.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry\n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        worst/largest values) of these features were computed for each image,\n        resulting in 30 features.  For inst

In [6]:
X, y = data.data, data.target

In [7]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [8]:
X.shape, y.shape

((569, 30), (569,))

In [9]:
np.unique(y)   # 0: benign, 1: malignant

array([0, 1])

In [10]:
feature_names = data.feature_names

In [11]:
feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [12]:
len(feature_names)
#feature_names.shape[0]

30

In [13]:
scaler = StandardScaler().fit(X)   
X_scaled = scaler.transform(X)     # features에 표준화를 적용하여 변수 변환

In [14]:
pca = PCA(n_components=0.90, random_state=0) # explained variance의 누적합이 90%가 되는 최소 개수의 주성분을 자동으로 선택  
X_pca = pca.fit_transform(X_scaled)          # random_state=0: 결과 재현성을 위해 seed를 고정

In [15]:
X_pca.shape # 행: 샘플의 수, 열: 선택된 주성분의 수

(569, 7)

In [16]:
n_pc = X_pca.shape[1]
n_pc

7

In [17]:
### OOB(Out-of-Bag) 스코어를 기준으로 Bagging 분류기의 주요 하이퍼파라미터를 Grid Search하기 위해 준비하는 부분 ###
results = []                        
param_grid = {                      ## key: Bagging Classifier의 주요 하이퍼파라미터들
    'n_estimators': [50, 100, 200],  # n_estimators: 앙상블에 사용할 트리의 개수   
    'max_samples': [0.7, 1.0],       # max_samples : 각 트리를 학습시킬 때 사용할 샘플의 비율
    'max_features': [0.7, 1.0]       # max_features: 각 트리 분할 시 고려할 특성의 비율
}                                   ## 하이퍼파라미터들의 조합의 수 = 3*2*2 = 12개

In [18]:
## param_grid에 정의된 하이퍼파라미터 조합을 전부 test해 보고
## 각 조합별로 OOB(Out-of-Bag) 스코어를 계산해
## results 리스트에 저장하는 그리드 탐색(grid search)을 수행한다

for n_est in param_grid['n_estimators']:                 # 'n_estimators': [50, 100, 200] 
    for max_s in param_grid['max_samples']:              # 'max_samples': [0.7, 1.0]
        for max_f in param_grid['max_features']:         # 'max_features': [0.7, 1.0]
            clf = BaggingClassifier(                    ## BaggingClassifier를 생성한다
                estimator=DecisionTreeClassifier(),      # 기본 학습기로 DecisionTreeClassifier를 사용한다
                n_estimators=n_est,
                max_samples=max_s,
                max_features=max_f,
                oob_score=True,                          # OOB 샘플을 사용하여 성능을 평가한다
                random_state=0,
                n_jobs=-1                                # 가능한 모든 CPU 코어를 활용해 병렬 학습한다
            )
            clf.fit(X_pca, y)
            results.append({
                'n_estimators': n_est,
                'max_samples': max_s,
                'max_features': max_f,
                'oob_score': clf.oob_score_               # 학습이 끝나면 clf.oob_score_ 속성에 저장된 OOB 정확도를 읽어온다
            })

In [19]:
vars(clf).keys()

dict_keys(['estimator', 'n_estimators', 'estimator_params', 'base_estimator', 'max_samples', 'max_features', 'bootstrap', 'bootstrap_features', 'oob_score', 'warm_start', 'n_jobs', 'random_state', 'verbose', 'n_features_in_', '_n_samples', 'classes_', 'n_classes_', 'estimator_', '_max_samples', '_max_features', 'estimators_', 'estimators_features_', '_seeds', 'oob_decision_function_', 'oob_score_'])

* oob_score(bool type parameter): 생성자 인자로 주어지는 설정값. 학습할 때 OOB 점수를 계산할 것인지 여부를 TRUE 또는 FALSE로 설정한다
* oob_socre_(float type attribute): fit() 호출 뒤, 실제로 계산된 OOB 점수를 담은 결과값. 분류 task: 정확도. 회귀 task: 수정된 결정계수.

In [20]:
type(results)

list

In [21]:
# 12개 하이퍼파라미터들의 조합에 대해 4개의 딕셔너리 key의 value를 보여준다
results_df = pd.DataFrame(results)
results_df

,n_estimators,max_samples,max_features,oob_score
0,50,0.7,0.7,0.942004
1,50,0.7,1.0,0.947276
2,50,1.0,0.7,0.942004
3,50,1.0,1.0,0.947276
4,100,0.7,0.7,0.952548
5,100,0.7,1.0,0.949033
6,100,1.0,0.7,0.957821
7,100,1.0,1.0,0.947276
8,200,0.7,0.7,0.954306
9,200,0.7,1.0,0.949033


In [22]:
type(results_df['oob_score'])

pandas.core.series.Series

In [23]:
best_idx = results_df['oob_score'].idxmax() # 이 Series에서 가장 큰 값을 갖는 인덱스를 반환한다
best_idx

6

In [24]:
# results_df에서 가장 높은 OOB 스코어를 기록한 행(row)을 딕셔너리 형태로 뽑아서 best_params에 저장한 후 출력한다
best_params = results_df.loc[best_idx].to_dict()
best_params

{'n_estimators': 100.0,
 'max_samples': 1.0,
 'max_features': 0.7,
 'oob_score': 0.9578207381370826}

In [25]:
# Grid Search로 찾아낸 최적 하이퍼파라미터를 이용해 최종 BaggingClassifier 모델을 다시 생성한다
best_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=int(best_params['n_estimators']),
    max_samples=best_params['max_samples'],
    max_features=best_params['max_features'],
    oob_score=True,
    random_state=0,
    n_jobs=-1
)

In [26]:
best_clf.fit(X_pca, y)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_features=0.7,
                  n_estimators=100, n_jobs=-1, oob_score=True, random_state=0)

In [27]:
### PC별 MDI 중요도 복원
pc_importances = np.zeros(n_pc) # n_pc: PCA로 축소된 후 남은 주성분의 수. PC별 누적 중요도를 저장할 배열을 만들고, 0으로 값 초기화
M = len(best_clf.estimators_)   # best_clf.estimators_: 학습된 트리 인스턴스들의 리스트. 그 리스트의 길이 M: 앙상블에 포함된 트리의 수

In [28]:
for tree, feats in zip(best_clf.estimators_, best_clf.estimators_features_):
    imp = tree.feature_importances_   # tree.feature_importances_: 적합된 트리가 len(feats)개의 축에 대해 계산한 MDI 중요도 벡터
                                      # 벡터 길이: len(feats). imp[i]는 feats[i]번째 주성분 축이 그 트리에서 얼마나 불순도 감소에 기여했는지
    pc_importances[feats] += imp      # feats에 대응하는 위치(pc_importances[feats])에 imp를 더해 누적한다                                    

In [29]:
pc_importances /= M
# 트리 수 M으로 나누어, 모든 트리에 걸친 평균 중요도로 정규화한다
# pc_importance[k]: 전체 앙상블에서 k번째 주성분이 받은 평균 MDI 중요도

In [30]:
## PCA 축 단위로 계산된 평균 중요도(pc_importances)를 다시 원본 변수(feature) 수준으로 되돌려, 각 원본 변수별 중요도를 근사하는 과정
loadings = np.abs(pca.components_.T) # (n_orig_features, n_pc) => 각 원본 변수가 어떤 주성분 축들과 얼마나 연관되는지가 열 방향으로 대응  
                                     # loadings[i,k]: 원본 변수 i가 주성분 k에 기여하는 정도
# pca.components_: pc의 로딩(loading) 벡터를 행 단위로 담고 있다
# pca.components_의 shape = (n_pc, n_orig_features)

In [31]:
orig_importances = loadings.dot(pc_importances) # 행렬곱(dot product)
# 각 원본 변수 i가 속한 주성분 축들에 나뉘어 있는 중요도를 로딩 비율에 따라 합산해서, 원본 변수 수준의 근사 MDI 중요도를 얻는다

In [32]:
orig_imp_df = pd.DataFrame({
    'feature': feature_names,
    'importance': orig_importances
}).sort_values('importance', ascending=False).reset_index(drop=True) # importance 열을 기준으로 내림차순 정렬
## reset_index(drop=True)
# 정렬 과정에서 뒤섞인 인덱스(원래 행 번호)를 0,1,2… 순서로 새로 매긴다
# drop=True 옵션으로 원래 인덱스 열은 버린다 

In [33]:
# best_params 딕셔너리에 저장된 최적 하이퍼파라미터 이름과 값 쌍을 pandas DataFrame으로 바꾼다
param_df = pd.DataFrame(
    best_params.items(),            
    columns=["Parameter","Value"] # 2개의 열 이름을 지정한다
)

In [34]:
print('Best hyperparameters (OOB 기준)')
print()
param_df

Best hyperparameters (OOB 기준)



,Parameter,Value
0,n_estimators,100.000000
1,max_samples,1.000000
2,max_features,0.700000
3,oob_score,0.957821


In [35]:
print('OOB Hyperparameter Results')
print()
results_df

OOB Hyperparameter Results



,n_estimators,max_samples,max_features,oob_score
0,50,0.7,0.7,0.942004
1,50,0.7,1.0,0.947276
2,50,1.0,0.7,0.942004
3,50,1.0,1.0,0.947276
4,100,0.7,0.7,0.952548
5,100,0.7,1.0,0.949033
6,100,1.0,0.7,0.957821
7,100,1.0,1.0,0.947276
8,200,0.7,0.7,0.954306
9,200,0.7,1.0,0.949033


In [36]:
print('Original Feature Importances (MDI+PCA)')
print()
orig_imp_df

Original Feature Importances (MDI+PCA)



,feature,importance
0,area error,0.188011
1,radius error,0.187305
2,perimeter error,0.184164
3,concave points error,0.179345
4,worst compactness,0.176944
5,worst concavity,0.172502
6,worst concave points,0.168535
7,mean smoothness,0.168262
8,mean concave points,0.166816
9,worst smoothness,0.164978
